In [1]:
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install matplotlib


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install fastapi


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install xgboost


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install -U scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from fastapi import FastAPI
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

In [7]:
app = FastAPI()

In [8]:
df = pd.read_csv("video_games_sales.csv")

In [9]:
df

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


transformé le name en valeur => (mot clé)
Package -> word cloud (compter les fréquences de mot dans la liste)
combien de genre / publisher / platform

In [10]:
df.describe()

,rank,year,na_sales,eu_sales,jp_sales,other_sales,global_sales
count,16598.000000,16327.000000,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000
mean,8300.605254,2006.406443,0.264667,0.146652,0.077782,0.048063,0.537441
std,4791.853933,5.828981,0.816683,0.505351,0.309291,0.188588,1.555028
min,1.000000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.010000
25%,4151.250000,2003.000000,0.000000,0.000000,0.000000,0.000000,0.060000
50%,8300.500000,2007.000000,0.080000,0.020000,0.000000,0.010000,0.170000
75%,12449.750000,2010.000000,0.240000,0.110000,0.040000,0.040000,0.470000
max,16600.000000,2020.000000,41.490000,29.020000,10.220000,10.570000,82.740000


In [11]:
# grouper par noms et calculer la moyenne des valeurs
df.groupby(["genre"]).agg({"eu_sales": "sum"}).reset_index()

,genre,eu_sales
0,Action,525.00
1,Adventure,64.13
2,Fighting,101.32
3,Misc,215.98
4,Platform,201.63
5,Puzzle,50.78
6,Racing,238.39
7,Role-Playing,188.06
8,Shooter,313.27
9,Simulation,113.38


In [12]:
(df.groupby(["publisher"]).count().sort_values("rank", ascending=False)['rank']).nlargest(50)

publisher
Electronic Arts                           1351
Activision                                 975
Namco Bandai Games                         932
Ubisoft                                    921
Konami Digital Entertainment               832
THQ                                        715
Nintendo                                   703
Sony Computer Entertainment                683
Sega                                       639
Take-Two Interactive                       413
Capcom                                     381
Atari                                      363
Tecmo Koei                                 338
Square Enix                                233
Warner Bros. Interactive Entertainment     232
Disney Interactive Studios                 218
Unknown                                    203
Eidos Interactive                          198
Midway Games                               198
505 Games                                  192
Microsoft Game Studios                     189
Acc

In [13]:
# grouper par année et genre et calculer les ventes totales
df_grouped = df.groupby(["year", "genre"])["eu_sales"].sum().reset_index()

# trouver l'index du genre le plus vendu pour chaque année
idx = df_grouped.groupby("year")["eu_sales"].idxmax()

# afficher le dataframe trié par année et le genre le plus vendu
print(df_grouped.loc[idx].sort_values("year"))

       year         genre  eu_sales
3    1980.0       Shooter      0.43
5    1981.0        Action      0.81
15   1982.0        Puzzle      0.58
22   1983.0      Platform      0.43
31   1984.0       Shooter      0.85
35   1985.0      Platform      3.77
40   1986.0        Action      1.66
48   1987.0      Fighting      0.65
54   1988.0      Platform      4.99
64   1989.0        Puzzle      3.61
69   1990.0      Platform      3.99
79   1991.0      Platform      1.19
88   1992.0     Adventure      2.81
103  1993.0      Platform      3.16
115  1994.0      Platform      5.73
127  1995.0      Platform      3.22
142  1996.0  Role-Playing     11.04
153  1997.0        Racing     10.82
159  1998.0        Action     11.90
178  1999.0  Role-Playing     11.17
183  2000.0        Action     10.84
195  2001.0        Action     19.25
207  2002.0        Action     27.01
219  2003.0        Action     20.88
238  2004.0  Role-Playing     16.73
243  2005.0        Action     21.95
265  2006.0        Sports   

-------------------------------------
# /!\ a supprimer /!\

- Nettoyer la data
- LabelEncoder
- OneHotEncoder -> créer des categories (collones) -> a regarder -> A PRENDRE
- split train test
- Xgboost Regression xgb(x.train, y.train) / xgb.predict(x.test, y.test)

- db.apply(callback, axis = 1) == js.map(callback)


# /!\ a supprimer /!\

-----------------------------------

In [38]:
df_filter = df.dropna()
df_filter

,rank,name,platform,year,genre,publisher,na_sales,eu_sales,jp_sales,other_sales,global_sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


##### recupérer les 50 éditeurs qui ont publié dans liste

In [15]:
df_publisher_no_unknown = df[df.publisher != 'Unknown']
df_publisher = (df_publisher_no_unknown.groupby(["publisher"]).count().sort_values("rank", ascending=False)['rank']).nlargest(50)
df_publisher

publisher
Electronic Arts                           1351
Activision                                 975
Namco Bandai Games                         932
Ubisoft                                    921
Konami Digital Entertainment               832
THQ                                        715
Nintendo                                   703
Sony Computer Entertainment                683
Sega                                       639
Take-Two Interactive                       413
Capcom                                     381
Atari                                      363
Tecmo Koei                                 338
Square Enix                                233
Warner Bros. Interactive Entertainment     232
Disney Interactive Studios                 218
Midway Games                               198
Eidos Interactive                          198
505 Games                                  192
Microsoft Game Studios                     189
Acclaim Entertainment                      184
D3P

In [37]:
df["publisher"].value_counts().head(50).index

Index(['Electronic Arts', 'Activision', 'Namco Bandai Games', 'Ubisoft',
       'Konami Digital Entertainment', 'THQ', 'Nintendo',
       'Sony Computer Entertainment', 'Sega', 'Take-Two Interactive', 'Capcom',
       'Atari', 'Tecmo Koei', 'Square Enix',
       'Warner Bros. Interactive Entertainment', 'Disney Interactive Studios',
       'Unknown', 'Midway Games', 'Eidos Interactive', '505 Games',
       'Microsoft Game Studios', 'D3Publisher', 'Acclaim Entertainment',
       'Vivendi Games', 'Codemasters', 'Idea Factory', 'Deep Silver',
       'Nippon Ichi Software', 'Zoo Digital Publishing',
       'Majesco Entertainment', 'LucasArts', 'Rising Star Games',
       'Hudson Soft', 'Banpresto', 'Bethesda Softworks', 'Crave Entertainment',
       'Atlus', 'Virgin Interactive', 'Infogrames', 'Ignition Entertainment',
       '5pb', 'Focus Home Interactive', 'Marvelous Interactive',
       'Empire Interactive', 'SquareSoft', 'Kadokawa Shoten', 'Destineer',
       'DTP Entertainment', 'GT I

In [34]:
(df.groupby(["publisher"]).count().sort_values("rank", ascending=False)['rank']).nlargest(50)

publisher
Electronic Arts                           1351
Activision                                 975
Namco Bandai Games                         932
Ubisoft                                    921
Konami Digital Entertainment               832
THQ                                        715
Nintendo                                   703
Sony Computer Entertainment                683
Sega                                       639
Take-Two Interactive                       413
Capcom                                     381
Atari                                      363
Tecmo Koei                                 338
Square Enix                                233
Warner Bros. Interactive Entertainment     232
Disney Interactive Studios                 218
Unknown                                    203
Eidos Interactive                          198
Midway Games                               198
505 Games                                  192
Microsoft Game Studios                     189
Acc

In [41]:
df_filter_not_unknown = df_filter[df_filter.publisher != 'Unknown']
df_filter_not_unknown['genre']

Action          3234
Sports          2297
Misc            1671
Role-Playing    1457
Shooter         1279
Adventure       1261
Racing          1222
Platform         870
Simulation       843
Fighting         835
Strategy         664
Puzzle           558
Name: genre, dtype: int64

In [18]:
y_df = df_filter_not_unknown["eu_sales"]
y_df

0        29.02
1         3.58
2        12.88
3        11.01
4         8.89
         ...  
16593     0.00
16594     0.00
16595     0.00
16596     0.01
16597     0.00
Name: eu_sales, Length: 16191, dtype: float64

In [19]:
x_df = df_filter_not_unknown[["platform","year","genre","publisher","na_sales","jp_sales","global_sales"]]
x_df

,platform,year,genre,publisher,na_sales,jp_sales,global_sales
0,Wii,2006.0,Sports,Nintendo,41.49,3.77,82.74
1,NES,1985.0,Platform,Nintendo,29.08,6.81,40.24
2,Wii,2008.0,Racing,Nintendo,15.85,3.79,35.82
3,Wii,2009.0,Sports,Nintendo,15.75,3.28,33.00
4,GB,1996.0,Role-Playing,Nintendo,11.27,10.22,31.37
...,...,...,...,...,...,...,...
16593,GBA,2002.0,Platform,Kemco,0.01,0.00,0.01
16594,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.01
16595,PS2,2008.0,Racing,Activision,0.00,0.00,0.01
16596,DS,2010.0,Puzzle,7G//AMES,0.00,0.00,0.01


In [20]:
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.3, random_state=123)

In [21]:
X_train

,platform,year,genre,publisher,na_sales,jp_sales,global_sales
7605,PS3,2010.0,Action,Konami Digital Entertainment,0.10,0.04,0.20
7817,DS,2010.0,Action,Activision,0.17,0.00,0.19
3340,XB,2005.0,Fighting,Electronic Arts,0.42,0.00,0.60
16031,PSP,2009.0,Adventure,TGL,0.00,0.01,0.01
4366,PS3,2008.0,Role-Playing,Atari,0.19,0.07,0.45
...,...,...,...,...,...,...,...
12512,N64,1998.0,Sports,GT Interactive,0.05,0.00,0.06
1360,SNES,1994.0,Action,Nintendo,0.57,0.71,1.42
11888,PS2,2004.0,Shooter,Global Star,0.04,0.00,0.07
16107,PSV,2014.0,Adventure,Idea Factory,0.00,0.01,0.01


In [22]:
y_test

2959     0.09
1218     0.41
5092     0.00
11477    0.00
7921     0.08
         ... 
2805     0.16
9119     0.03
15654    0.01
9780     0.03
723      0.61
Name: eu_sales, Length: 4858, dtype: float64

In [33]:
model = XGBRegressor()
model.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:platform: object, genre: object, publisher: object